<a href="https://colab.research.google.com/github/PrinceVhelbreiBarbarona/Final-Project-CS2./blob/main/CS2FinalProject_NB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Scroll **all the way down** for the actual program

In [1]:
import requests
import json
url = "https://raw.githubusercontent.com/PrinceVhelbreiBarbarona/Final-Project-CS2./refs/heads/main/data/hospital.json"

response = requests.get(url)
patient = response.json()
print(patient)

[{'patient_id': 1, 'name': 'John Cruz', 'age': 45, 'gender': 'Male', 'visits': [{'visit_date': '2025-08-12', 'diagnosis': 'Diabetes Type II', 'treatment': 'Metformin 500mg daily', 'doctor': 'Dr. Santos', 'notes': 'Advised low-carb diet and regular exercise.'}, {'visit_date': '2025-09-10', 'diagnosis': 'Diabetes Type II', 'treatment': 'Increased Metformin dosage to 850mg', 'doctor': 'Dr. Santos', 'notes': 'Blood sugar levels still above target.'}]}, {'patient_id': 2, 'name': 'Ana Gomez', 'age': 32, 'gender': 'Female', 'visits': [{'visit_date': '2025-08-15', 'diagnosis': 'Hypertension Stage 1', 'treatment': 'Losartan 50mg daily', 'doctor': 'Dr. Dela Cruz', 'notes': 'Encouraged lifestyle modifications.'}, {'visit_date': '2025-09-05', 'diagnosis': 'Hypertension Stage 1', 'treatment': 'Continue Losartan 50mg', 'doctor': 'Dr. Dela Cruz', 'notes': 'BP improved; maintain current plan.'}]}, {'patient_id': 3, 'name': 'Luis Ramos', 'age': 60, 'gender': 'Male', 'visits': [{'visit_date': '2025-08-1

In [3]:
!pip install firebase_admin

In [4]:
import firebase_admin
from firebase_admin import credentials, db
cred = credentials.Certificate("/content/cs2finalprojectg5-db-firebase-adminsdk-fbsvc-00dcf55902.json")
firebase_admin.initialize_app(cred, {
"databaseURL": "https://cs2finalprojectg5-db-default-rtdb.asia-southeast1.firebasedatabase.app/"
})
print("Firebase connected successfully!")

Firebase connected successfully!


In [5]:
import json
print("Patient data ready for upload.")
ref = db.reference("patient")
for p_data in patient:
    ref.child(str(p_data["patient_id"])).set(p_data)
print("Data uploaded successfully!")

Patient data ready for upload.
Data uploaded successfully!


In [7]:
def count_patient_visits(patient_id):
    """
    Counts and displays the total number of visits for a given patient ID.
    Fetches patient data from the 'patient' Firebase reference.
    """
    patient_ref = db.reference(f"patient/{patient_id}")
    patient_data = patient_ref.get()

    if patient_data:
        patient_name = patient_data.get('name', 'Unknown Patient')
        visits = patient_data.get('visits', [])
        num_visits = len(visits)
        print(f"Patient: {patient_name} (ID: {patient_id})")
        print(f"Total number of visits: {num_visits}")
    else:
        print(f"Patient with ID {patient_id} not found.")

print("count_patient_visits function defined.")

count_patient_visits function defined.


In [10]:
def list_doctors_consulted(patient_id):
    """
    Lists all unique doctors consulted by a patient with the given ID.
    Fetches patient data from the 'patient' Firebase reference.
    """
    patient_ref = db.reference(f"patient/{patient_id}")
    patient_data = patient_ref.get()

    if patient_data:
        patient_name = patient_data.get('name', 'Unknown Patient')
        visits = patient_data.get('visits', [])

        unique_doctors = set()
        for visit in visits:
            doctor = visit.get('doctor')
            if doctor:
                unique_doctors.add(doctor)

        print(f"\nPatient: {patient_name} (ID: {patient_id})")
        if unique_doctors:
            print("Doctors Consulted:")
            for doctor in sorted(list(unique_doctors)): # Sort for consistent output
                print(f"- {doctor}")
        else:
            print("No doctors found for this patient.")
    else:
        print(f"Patient with ID {patient_id} not found.")

print("list_doctors_consulted function defined.")

list_doctors_consulted function defined.


In [13]:
def identify_diagnoses(patient_id):
    """
    Identifies and displays all unique diagnoses for a given patient ID.
    Fetches patient data from the 'patient' Firebase reference.
    """
    patient_ref = db.reference(f"patient/{patient_id}")
    patient_data = patient_ref.get()

    if patient_data:
        patient_name = patient_data.get('name', 'Unknown Patient')
        visits = patient_data.get('visits', [])

        unique_diagnoses = set()
        for visit in visits:
            diagnosis = visit.get('diagnosis')
            if diagnosis:
                unique_diagnoses.add(diagnosis)

        print(f"\nPatient: {patient_name} (ID: {patient_id})")
        if unique_diagnoses:
            print("Unique Diagnoses:")
            for diagnosis in sorted(list(unique_diagnoses)): # Sort for consistent output
                print(f"- {diagnosis}")
        else:
            print("No diagnoses found for this patient.")
    else:
        print(f"Patient with ID {patient_id} not found.")

print("identify_diagnoses function defined.")

identify_diagnoses function defined.


In [21]:
def analyze_demographic_doctor_preference():
    """
    Analyzes demographic doctor preferences by categorizing patients by age and gender
    and identifying the most frequently consulted doctors for each group.
    Fetches all patient data from the 'patient' Firebase reference.
    """
    all_patients_data_raw = db.reference('patient').get()

    if not all_patients_data_raw:
        print("No patient data found in Firebase.")
        return

    # Normalize data to always be a list of valid patient dictionaries, filtering out None values
    patients_to_process = []
    if isinstance(all_patients_data_raw, dict):
        # If Firebase returned a dictionary (patient_id: patient_data),
        # take the values (patient_data dictionaries) and filter None.
        patients_to_process = [p for p in all_patients_data_raw.values() if p is not None]
    elif isinstance(all_patients_data_raw, list):
        # If Firebase returned a list of patient_data dictionaries directly, filter None.
        patients_to_process = [p for p in all_patients_data_raw if p is not None]
    else:
        print("Unexpected data format from Firebase: expected dict or list.")
        return

    # Initialize dictionaries to store doctor consultation counts for demographics
    gender_doctor_counts = {
        'Male': {}, 'Female': {}
    }
    age_group_doctor_counts = {
        'Young (0-25)': {}, 'Adult (26-55)': {}, 'Senior (56+)': {}
    }

    # Iterate through each patient in the retrieved data
    for patient_data in patients_to_process:
        age = patient_data.get('age')
        gender = patient_data.get('gender')
        visits = patient_data.get('visits', [])

        if age is None or gender is None:
            continue # Skip patients with missing age or gender

        # Determine age group
        age_group = ''
        if 0 <= age <= 25:
            age_group = 'Young (0-25)'
        elif 26 <= age <= 55:
            age_group = 'Adult (26-55)'
        elif age >= 56:
            age_group = 'Senior (56+)'

        # Process visits for each patient
        for visit in visits:
            doctor = visit.get('doctor')
            if doctor:
                # Update gender-based counts
                if gender in gender_doctor_counts:
                    gender_doctor_counts[gender][doctor] = gender_doctor_counts[gender].get(doctor, 0) + 1

                # Update age group-based counts
                if age_group in age_group_doctor_counts:
                    age_group_doctor_counts[age_group][doctor] = age_group_doctor_counts[age_group].get(doctor, 0) + 1

    print("\n--- Demographic Doctor Preference Analysis ---")

    # Function to find and print preferred doctors for a given demographic group
    def print_preferred_doctors(demographic_name, doctor_counts_dict):
        if not doctor_counts_dict:
            print(f"No doctor preferences found for {demographic_name}.")
            return

        max_count = 0
        preferred_doctors = []
        for doctor, count in doctor_counts_dict.items():
            if count > max_count:
                max_count = count
                preferred_doctors = [doctor]
            elif count == max_count:
                preferred_doctors.append(doctor)

        if preferred_doctors:
            print(f"\nDemographic: {demographic_name}")
            print(f"Most preferred doctor(s): {', '.join(sorted(preferred_doctors))} (Consultations: {max_count})")
        else:
            print(f"No doctor preferences found for {demographic_name}.")

    # Display results for gender
    print("\nGender-based Preferences:")
    print_preferred_doctors('Male', gender_doctor_counts['Male'])
    print_preferred_doctors('Female', gender_doctor_counts['Female'])

    # Display results for age groups
    print("\nAge Group-based Preferences:")
    print_preferred_doctors('Young (0-25)', age_group_doctor_counts['Young (0-25)'])
    print_preferred_doctors('Adult (26-55)', age_group_doctor_counts['Adult (26-55)'])
    print_preferred_doctors('Senior (56+)', age_group_doctor_counts['Senior (56+)'])

print("analyze_demographic_doctor_preference function defined.")

analyze_demographic_doctor_preference function defined.


In [31]:
def analyze_doctor_prescription_habits(doctor_name):
    """
    Analyzes the prescription habits of a specific doctor.
    Prompts the user for a doctor's name, fetches all patient data from Firebase,
    and then analyzes the treatments prescribed by the specified doctor across all their visits
    to identify common prescription patterns or frequently used medications.
    """
    all_patients_data_raw = db.reference('patient').get()

    if not all_patients_data_raw:
        print("No patient data found in Firebase.")
        return

    # Normalize data to always be a list of valid patient dictionaries, filtering out None values
    patients_to_process = []
    if isinstance(all_patients_data_raw, dict):
        patients_to_process = [p for p in all_patients_data_raw.values() if p is not None]
    elif isinstance(all_patients_data_raw, list):
        patients_to_process = [p for p in all_patients_data_raw if p is not None]
    else:
        print("Unexpected data format from Firebase: expected dict or list.")
        return

    prescription_counts = {}

    for patient_data in patients_to_process:
        visits = patient_data.get('visits', [])
        for visit in visits:
            if visit.get('doctor') == doctor_name:
                treatment = visit.get('treatment')
                if treatment:
                    prescription_counts[treatment] = prescription_counts.get(treatment, 0) + 1

    print(f"\n--- Prescription Habits for Dr. {doctor_name} ---")
    if not prescription_counts:
        print(f"No prescription data found for Dr. {doctor_name}.")
    else:
        # Sort by frequency (descending) then by treatment name (alphabetical)
        sorted_prescriptions = sorted(
            prescription_counts.items(),
            key=lambda item: (-item[1], item[0])
        )
        print(f"Common prescription patterns for Dr. {doctor_name}:")
        for treatment, count in sorted_prescriptions:
            print(f"- {treatment}: {count} times")


In [25]:
def normalize_doctor_name(name):
    """
    Changes the users input to be normalized by changing it to lower case and removing dr prefixes
    """
    if name is None:
        return None
    normalized_name = name.lower()
    if normalized_name.startswith('dr.'):
        normalized_name = normalized_name[3:].strip()
    elif normalized_name.startswith('dr '):
        normalized_name = normalized_name[2:].strip()
    return normalized_name


normalize_doctor_name helper function defined.


In [30]:
def analyze_doctor_prescription_habits():
    """
    Analyzes the prescription habits of a specific doctor.
    Prompts the user for a doctor's name, fetches all patient data from Firebase,
    and then analyzes the treatments prescribed by the specified doctor across all their visits
    to identify common prescription patterns or frequently used medications.
    """
    doctor_name_input = input("Enter Doctor's Name (e.g., Dr. Santos): ")
    if not doctor_name_input:
        print("Doctor's name cannot be empty.")
        return

    normalized_input_doctor_name = normalize_doctor_name(doctor_name_input)
    if normalized_input_doctor_name is None or normalized_input_doctor_name == '':
        print("Invalid doctor's name provided.")
        return

    all_patients_data_raw = db.reference('patient').get()

    if not all_patients_data_raw:
        print("No patient data found in Firebase.")
        return

    patients_to_process = []
    if isinstance(all_patients_data_raw, dict):
        patients_to_process = [p for p in all_patients_data_raw.values() if p is not None]
    elif isinstance(all_patients_data_raw, list):
        patients_to_process = [p for p in all_patients_data_raw if p is not None]
    else:
        print("Unexpected data format from Firebase: expected dict or list.")
        return

    prescription_counts = {}

    for patient_data in patients_to_process:
        visits = patient_data.get('visits', [])
        for visit in visits:
            doctor_from_db = visit.get('doctor')
            normalized_doctor_from_db = normalize_doctor_name(doctor_from_db)

            if normalized_doctor_from_db == normalized_input_doctor_name:
                treatment = visit.get('treatment')
                if treatment:
                    prescription_counts[treatment] = prescription_counts.get(treatment, 0) + 1

    print(f"\n--- Prescription Habits for {doctor_name_input} ---")
    if not prescription_counts:
        print(f"No prescription data found for {doctor_name_input}.")
    else:
        # Sort by frequency (descending) then by treatment name (alphabetical)
        sorted_prescriptions = sorted(
            prescription_counts.items(),
            key=lambda item: (-item[1], item[0])
        )
        print(f"Common prescription patterns for {doctor_name_input}:")
        for treatment, count in sorted_prescriptions:
            print(f"- {treatment}: {count} times")


Below this is the actual program just run it and it will work. *(i hope)*

In [34]:
def main_program():
    while True:
        print("\n--- Hospital Data Analysis Menu ---")
        print("1. Count patient visits")
        print("2. List doctors consulted by a patient")
        print("3. Identify diagnoses for a patient")
        print("4. Analyze demographic doctor preference")
        print("5. Analyze doctor prescription habits")
        print("6. Exit program")
        print("-----------------------------------")

        choice = input("Enter your choice (1-6): ")

        if choice == '1':
            try:
                patient_id_input = input("Enter Patient ID: ")
                patient_id = int(patient_id_input)
                count_patient_visits(patient_id)
            except ValueError:
                print("Invalid Patient ID. Please enter a number.")
        elif choice == '2':
            try:
                patient_id_input = input("Enter Patient ID: ")
                patient_id = int(patient_id_input)
                list_doctors_consulted(patient_id)
            except ValueError:
                print("Invalid Patient ID. Please enter a number.")
        elif choice == '3':
            try:
                patient_id_input = input("Enter Patient ID: ")
                patient_id = int(patient_id_input)
                identify_diagnoses(patient_id)
            except ValueError:
                print("Invalid Patient ID. Please enter a number.")
        elif choice == '4':
            analyze_demographic_doctor_preference()
        elif choice == '5':
            analyze_doctor_prescription_habits()
        elif choice == '6':
            print("Exiting program. Goodbye!")
            break
        else:
            print("Invalid choice. Please enter a number between 1 and 6.")

main_program()



--- Hospital Data Analysis Menu ---
1. Count patient visits
2. List doctors consulted by a patient
3. Identify diagnoses for a patient
4. Analyze demographic doctor preference
5. Analyze doctor prescription habits
6. Exit program
-----------------------------------
Enter your choice (1-6): 6
Exiting program. Goodbye!
